In [ ]:
# 加载环境变量，如果你用的是OpenAI key就用这个方案！去掉注释即可！后面定义本地model的时候，把它注释
# import dotenv
# %load_ext dotenv
# %dotenv

In [2]:
# import nest_asyncio
# nest_asyncio.apply()

In [62]:
# 安装所有依赖
!pip install python-dotenv ipykernel llama-index nest_asyncio
!pip install timm
!pip install llama-index-llms-huggingface
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor

Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple
Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple
Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple
  Using cached https://mirrors.tuna.tsinghua.edu.cn/pypi/web/packages/f0/62/9ebaf1fdd3d3c737a8814f9ae409d4ac04bc93b26a46a7dab456bb7e16f8/accelerate-0.31.0-py3-none-any.whl (309 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.17.0
    Uninstalling accelerate-0.17.0:
      Successfully uninstalled accelerate-0.17.0
Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple
  Using cached https://mirrors.tuna.tsinghua.edu.cn/pypi/web/packages/76/2c/bd95032aeb087b0706596af0a4518c4bfe0439a1bb149048ece18b617766/sentence_transformers-2.7.0-py3-none-any.whl (171 kB)
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.0.1
    Uninstalling sentence-transformers-3.0.1:
      Succes

In [1]:
from llama_index.core import SimpleDirectoryReader

# 加载PDF数据，这里雄哥以卖油翁为例，可以改为自己的任意数据放在datasets文件夹即可，或改为指定的绝对路径
documents = SimpleDirectoryReader(input_files=["./datasets/maiyouweng.pdf"]).load_data()

In [2]:
from llama_index.core.node_parser import SentenceSplitter

# 把文本分块，chunk_size为1024，每一块的大小
splitter = SentenceSplitter(chunk_size=1024)
# 创建文档的节点
nodes = splitter.get_nodes_from_documents(documents)

In [3]:
# 这里可以打印每一个块的信息！
node_metadata = nodes[1].get_content(metadata_mode=True)
print(node_metadata)

page_label: 1
file_name: maiyouweng.pdf
file_path: datasets\maiyouweng.pdf
file_type: application/pdf
file_size: 429687
creation_date: 2024-06-13
last_modified_date: 2024-04-29

尔，同“耳”，相当于“罢了”。
12.忿（fèn）然：气愤的样子。然，作形容词或者副词的词尾，相当于“的”或“地”。
13.安：怎么。轻吾射：看轻我射箭（的本领）。轻，作动词用。
14.以我酌（zhuó）油知之：凭我倒油（的经验）知道这个（道理）。以，凭、靠。
酌，斟酒，这里指倒油。之，指射箭也是凭手熟的道理。
15.覆：盖。
16.徐：慢慢地。
17.杓：同“勺”。
18.沥之：注入葫芦。沥，注。之，指葫芦。
19.惟：意为只，不过。
20.遣之：让他走。遣，打发。


In [4]:
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

C:\Users\ouhui\.conda\envs\xuehai3\lib\site-packages\pydantic\_internal\_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [5]:
# 加载本地的qwen2-7b，你本地的模型放在哪就直接改路径
llm = HuggingFaceLLM(
    tokenizer_name="G:\\models\\Qwen2-7B-Instruct",
    model_name="G:\\models\\Qwen2-7B-Instruct",
    device_map="auto",
    tokenizer_kwargs={"trust_remote_code": True},
    model_kwargs={"trust_remote_code": True},
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# 调用本地bce-embedding-base_v1作为embedding模型

embed_args = {
    'model_name': 'hkunlp/instructor-base', 
    'max_length': 512, 
    'embed_batch_size': 32, 
    'device': 'cuda'
    }
embed_model = HuggingFaceEmbedding(**embed_args)

C:\Users\ouhui\.conda\envs\xuehai3\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [7]:
from llama_index.core import Settings
# 把加载的模型添加到环境设置中，定义全局模型

Settings.llm = llm
Settings.embed_model = embed_model

In [8]:
from llama_index.core import SummaryIndex, VectorStoreIndex

# 创建数据摘要索引
summary_index = SummaryIndex(nodes)
# 创建矢量存储索引
vector_index = VectorStoreIndex(nodes)

In [9]:
# 把这些索引，转换为工具，后面雄哥就可以调用这些工具
# 摘要查询引擎
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)

# 向量查询引擎
vector_query_engine = vector_index.as_query_engine()

In [10]:
from llama_index.core.tools import QueryEngineTool

# 定义summary_tool的工具
summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "适用于生成与卖油翁课堂相关的摘要问题。"
    ),
)

# 定义vector_tool的工具
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "适用于检索卖油翁常规上下文的问题。"
    ),
)

In [16]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

# 定义所有的tools
query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [17]:
import nest_asyncio
nest_asyncio.apply()

In [18]:
response = query_engine.query("卖油翁课堂有什么内容?")
print(str(response))

Selecting query engine 0: 这个工具提供了关于《卖油翁》的快速总结和关键信息，直接回答了'卖油翁课堂有什么内容?'的问题。.
Selecting query engine 1: 虽然这个工具能够深入分析人物关系和重要概念，但它并未直接回答问题中的具体内容。.
《卖油翁》课堂内容涉及以下几个方面：

1. **教材解析**：讲解教材版本、章节位置，并强调文章的趣味性、文学价值和教育意义。

2. **作品原文**：提供文章的原文，概述了陈尧咨擅长射箭以及卖油翁通过倒油技艺展示的“熟能生巧”道理。

3. **词句注释**：对文章中的关键术语进行解释，例如“公”、“自矜”、“家圃”等。

4. **白话译文**：将原文翻译为现代汉语，以便更好地理解文章内容。

5. **重点句子解析**：选择并深入分析文章中的关键句子，以揭示文章的主题。

6. **字词释义**：详细解析一词多义、代词、介词和连词的用法，以及“尔”等通假字的含义。

课堂还涵盖了：

- **文言词汇**：解释关键词的含义和用法，如“以”、“其”、“之”。

- **语法结构**：分析句式特点，包括省略句、倒装句，以及特定句式的使用，如“吾射不亦精乎？”的表达方式


In [19]:
print(len(response.source_nodes))

14


In [25]:
response = query_engine.query("卖油翁的课后如何延展？")
print(str(response))

Selecting query engine 0: 这个工具可以帮助孩子更好地理解和记忆课文内容，因此是关于如何延展《卖油翁》这堂课的有效方法。.
Selecting query engine 1: 虽然这个工具能提供更深入的理解，但主要侧重于分析人物关系和重要概念，可能不是最直接的方法来延展这堂课。.
《卖油翁》的课后延展可以从多个角度进行，旨在深化学生对文章的理解、提高其语文素养、促进思维发展，并培养实际生活中的技能与品质。以下是几个具体的延展方向：

1. **深入理解主题与人物**：引导学生分析陈尧咨和卖油翁的人物性格，以及文章所传达的“熟能生巧”、“谦虚使人进步”等主题。鼓励学生思考这些概念在现实生活中的应用，如在学习新技能、解决日常问题时如何体现“熟能生巧”。

2. **拓展阅读与比较**：推荐其他类似主题的文学作品，如《愚公移山》、《铁杵成针》等，让学生通过比较不同故事中的主题与情节，加深对“熟能生巧”这一理念的理解，同时培养批判性思维和跨文化理解能力。

3. **实践操作与亲身体验**：鼓励学生尝试一些需要技巧和练习的活动，如烹饪、乐器演奏、书法等。通过亲身实践，体验技能提升的过程，记录并反思自己的感受与心得，从而更好地理解“熟能生巧”的意义。

4. **讨论与分享**：组织课堂讨论，让学生分享


In [26]:
print(len(response.source_nodes))

14
